In [40]:

def generate_tables(gf_table_size,polynomial):
    global gf_table_size
    polynomial = 0x11d # 2**8 + 2**4 + 2**3 + 2**2 + 1 =  1 0001 1101 binary
    s = 8
    gf_table_size = 1 << s # 1<<8 = 0x100 = 256 = 2**8

    # Initialize 
    gflog = gf_table_size * [0]
    gfilog = gf_table_size * [0]
    #print("gflog-->",gflog)

    b = 1
    for i in range(0, gf_table_size):
        gflog[b] = i    # & 255
        gfilog[i] = b   # & 255 
        b <<= 1
        # Following line could also be expressed as "if b >= gf_table_size"
        if b & gf_table_size:
            b ^= polynomial

    gflog[1] = 0; # Q for students: Why do you think is this needed? Try printing it out and see.
    return (gflog, gfilog)

# Note gf_table_size is now a global variable. Consider writing a helper function to control access to it.


SyntaxError: name 'gf_table_size' is parameter and global (<ipython-input-40-7750e1f8d6f7>, line 2)

In [1]:
def dump_table(caption, tab):
    item = 0
    print("--- {} ---".format(caption))
    for i in tab:
        print("0x{:02x}, ".format(i), end="")
        item += 1
        if item % 16 == 0:
            item = 0
            print()
    print("")

In [2]:
def qdump(tab):
    item = 0
    for i in tab:
        print("0x{:02x}, ".format(i), end="")
        item += 1
        if item % 16 == 0:
            item = 0
            print()
    print("")

In [5]:
#(gflog, gfilog) = generate_tables()
#Sample of desired new interface:
(gflog, gfilog) = generate_tables(8,0x11d)

In [7]:
#You may uncomment this code for testing.
dump_table("gflog",gflog)
dump_table("gfilog",gfilog)

--- gflog ---
0x00, 0x00, 0x01, 0x19, 0x02, 0x32, 0x1a, 0xc6, 0x03, 0xdf, 0x33, 0xee, 0x1b, 0x68, 0xc7, 0x4b, 
0x04, 0x64, 0xe0, 0x0e, 0x34, 0x8d, 0xef, 0x81, 0x1c, 0xc1, 0x69, 0xf8, 0xc8, 0x08, 0x4c, 0x71, 
0x05, 0x8a, 0x65, 0x2f, 0xe1, 0x24, 0x0f, 0x21, 0x35, 0x93, 0x8e, 0xda, 0xf0, 0x12, 0x82, 0x45, 
0x1d, 0xb5, 0xc2, 0x7d, 0x6a, 0x27, 0xf9, 0xb9, 0xc9, 0x9a, 0x09, 0x78, 0x4d, 0xe4, 0x72, 0xa6, 
0x06, 0xbf, 0x8b, 0x62, 0x66, 0xdd, 0x30, 0xfd, 0xe2, 0x98, 0x25, 0xb3, 0x10, 0x91, 0x22, 0x88, 
0x36, 0xd0, 0x94, 0xce, 0x8f, 0x96, 0xdb, 0xbd, 0xf1, 0xd2, 0x13, 0x5c, 0x83, 0x38, 0x46, 0x40, 
0x1e, 0x42, 0xb6, 0xa3, 0xc3, 0x48, 0x7e, 0x6e, 0x6b, 0x3a, 0x28, 0x54, 0xfa, 0x85, 0xba, 0x3d, 
0xca, 0x5e, 0x9b, 0x9f, 0x0a, 0x15, 0x79, 0x2b, 0x4e, 0xd4, 0xe5, 0xac, 0x73, 0xf3, 0xa7, 0x57, 
0x07, 0x70, 0xc0, 0xf7, 0x8c, 0x80, 0x63, 0x0d, 0x67, 0x4a, 0xde, 0xed, 0x31, 0xc5, 0xfe, 0x18, 
0xe3, 0xa5, 0x99, 0x77, 0x26, 0xb8, 0xb4, 0x7c, 0x11, 0x44, 0x92, 0xd9, 0x23, 0x20, 0x89, 0x2e, 
0x37, 0x3f, 0xd1

In [10]:
gf_table_size

NameError: name 'gf_table_size' is not defined

In [13]:
#You may uncomment this code to dump log/antilog columns for testing.
print ("gflog(n) n   gfilog(n)")
#num_to_print=min(64,gf_table_size)
#for index in range(num_to_print):
#    print ("{:02x}       {:02x}  {:02x}".format(gflog[index],index, gfilog[index]))

gflog(n) n   gfilog(n)


In [14]:
# Galois Field Mathematics

In [14]:
# Addition
# Under a Galois Field 2^n, addition is always a simple XOR.
#The below code doesn't rely on specific field size (2^8 or 2^n). So code remains same
def gf_add(*args):
    result = 0
    for arg in args:
        result ^= arg

    return result

In [15]:
# Multiplication
#Specific to GF(2^8)
def gf_mul(a, b):
    global gflog
    global gfilog

    if a == 0 or b == 0:
        return 0
    else:
        return gfilog[(gflog[a] + gflog[b]) % 255]

In [16]:
# Multiplication
#Specific to GF(2^n)
def gf_mul(a, b, gf_table_size):
    global gflog
    global gfilog
    if a == 0 or b == 0:
        return 0
    else:
        return gfilog[(gflog[a] + gflog[b]) % gf_table_size]

In [17]:
# Subtraction (Division helper) 
#Specific to GF(2^8)
def sub_gf8(a, b):
    if a > b:
        return a - b
    else:
        return (255 - (0 - (a - b)))  # Weird way to do modulus. 


In [18]:
# Subtraction (Division helper) 
#Specific to GF(2^n)
def sub_gf(a, b, gf_table_size):
    if a >= b:
        return a ^ b
    else:
        return (a + gf_table_size) ^ b

In [19]:
# Division
#Specific to GF(2^8) because of sub_gf8 reliance
def gf_div(a, b):
    global gfilog
    global gflog

    return gfilog[sub_gf8(gflog[a], gflog[b])]

In [20]:
# Division
#Specific to GF(2^n) 
def gf_div(a, b, gf_table_size):
    if a == 0:
        return 0  # Division by 0 is 0 in any finite field
    elif b == 0:
        raise ZeroDivisionError("Division by zero is undefined in finite fields.")

    global gflog
    global gfilog

    return gfilog[(gflog[a] - gflog[b] + gf_table_size) % gf_table_size]

In [21]:
commentedout='''#Print a portion of the gf Multiplication Table
startx=0x0
starty=0x0

numrows=min(32,gf_table_size)
numcols=min(32,gf_table_size)

endx=startx+numcols
endy=starty+numrows

print ("   |", end="")
for x in range(startx,endx):
    print(" {:02x}".format(x), end="")
print("")
print ("---+", end="")
for x in range(startx,endx):
    print("---",end="")
print("")

for y in range(starty,endy):
    print("{:02x} |".format(y), end="")
    for x in range(startx,endx):
        print(" {:02x}".format(gf_mul(x,y)), end="")
    print("")
'''       


In [22]:
# initialize data on all the data drives

numdrives = 4
blocksize = 5
drive=[0] * numdrives
drive[0] = [ ord('f'), ord('i'), ord('r'), ord('s'), ord('t') ]
drive[1] = [ ord('s'), ord('e'), ord('c'), ord('n'), ord('d') ]
drive[2] = [ ord('t'), ord('h'), ord('i'), ord('r'), ord('d') ]
drive[3] = [ ord('f'), ord('o'), ord('r'), ord('t'), ord('h') ]

In [23]:
# Replace above with, e.g.:
#def initialize_data_drives(int numdrives, int blocks_per_drive, random=True):
#   code...
# Where random=false leaves all drives populated with zeroes, 
#   and random=true initializes all data blocks with random bytes.

# You may leave numdrives and blocksize as global variables, or write helper functions to manage them.
def initialize_data_drives(numdrives, blocks_per_drive):
    data_drives = []
    for _ in range(numdrives):
        drive = [random.randint(0, 255) for _ in range(blocks_per_drive)]
        data_drives.append(drive)
    return data_drives

In [24]:
# rezero P and Q parity drives.
def zeroP(blocks_per_drive):
    return [0] * blocks_per_drive
#driveP = [0] * blocksize
#driveQ = [0] * blocksize
#driveP = [0] * blocks_per_drive
#driveQ = [0] * blocks_per_drive
def zeroQ(blocks_per_drive):
    return [0] * blocks_per_drive

# Replace above with zeroP and zeroQ functions.

In [27]:
#Calling functions to create zero-filled P and Q drives.
#zeroP(blocks_per_drive)
#zeroQ(blocks_per_drive)

In [25]:
# initialize P and Q parity drives

def generateP():
    for i in range(0, blocksize):
        driveP[i] = 0
        for d in range (0,numdrives):
            driveP[i] = driveP[i] ^ drive[d][i]
            
    return


In [26]:
def generateQ():
    for i in range(0,blocksize):
        driveQ[i] = 0
        for d in range(0,numdrives):
            driveQ[i] = gf_add(gf_mul(gfilog[d],drive[d][i]), driveQ[i])
    return

In [27]:
#generateP()
#generateQ()
# Also support (at least) single-stripe reconstruction

def generateP(data_drives, failed_drives):
    # Implement P generation logic for a single stripe
    for i in range(0, blocksize):
        driveP[i] = 0
        for d in range (0,numdrives):
            driveP[i] = driveP[i] ^ drive[d][i]
            
    return
    
def generateQ(data_drives, failed_drives):
    # Implement Q generation logic for a single stripe
    for i in range(0,blocksize):
        driveQ[i] = 0
        for d in range(0,numdrives):
            driveQ[i] = gf_add(gf_mul(gfilog[d],drive[d][i]), driveQ[i])
    return


In [34]:
generateP (stripe=2)
#generateQ (stripe=2)

TypeError: generateP() got an unexpected keyword argument 'stripe'

In [28]:
def dump_all_drives():
    for d in range(0,numdrives):
        s = "drive{}".format(d)
        dump_table(s,drive[d])
    dump_table("P parity",driveP)
    dump_table("Q parity",driveQ)
    return



In [29]:
dump_all_drives()

--- drive0 ---
0x66, 0x69, 0x72, 0x73, 0x74, 
--- drive1 ---
0x73, 0x65, 0x63, 0x6e, 0x64, 
--- drive2 ---
0x74, 0x68, 0x69, 0x72, 0x64, 
--- drive3 ---
0x66, 0x6f, 0x72, 0x74, 0x68, 


NameError: name 'driveP' is not defined

In [35]:
#Fail and Reconstruct P Parity
driveP = [0] * blocksize
dump_table("driveP",driveP)
generateP()
dump_table("driveP",driveP)

--- driveP ---
0x00, 0x00, 0x00, 0x00, 0x00, 


TypeError: generateP() missing 2 required positional arguments: 'data_drives' and 'failed_drives'

In [23]:
#Show difference between gflog, gfilog, and inverse gfilog
code='''
print ("gflog(n) n   gfilog(n)  1/gfilog(n)")
for index in range(32):
    print ("{:02x}       {:02x}  {:02x}         {:02x}".format(gflog[index],index, gfilog[index],gf_div(1,gfilog[index])))
'''    

In [24]:
# Add fail and reconstruct here:
#def fail_drive(drive)
def fail_drive(drive_num, data_drives, failed_drives):
    if drive_num < len(data_drives):
        data_drives[drive_num] = [0] * len(data_drives[0])
        failed_drives.add(drive_num)

In [ ]:
def reconstruct_drive(drive_num, data_drives, failed_drives):
    if drive_num in failed_drives:
        failed_drives.remove(drive_num)
        # Implement drive reconstruction logic here
        # ...


In [25]:
# Fail and reconstruct drive[2] and the P Parity
drive[2] = [0] * blocksize
driveP = [0] * blocksize
print ("after failing drives 2 and P")
dump_all_drives()
print ()

# Rebuild drive[2]
for i in range(0,blocksize):
    partialQ = 0  # a.k.a. Q syndrome
    for d in range(0,numdrives):
        partialQ = gf_add(gf_mul(gfilog[d],drive[d][i]),
                          partialQ)
    partialQ = gf_add(partialQ,driveQ[i])
    
    # Divide the partialQ by the GF multiplier for the missing drive (multiply by the inverse)
    inverse = gf_div(1,gfilog[2])
    drive[2][i] = gf_mul(partialQ,inverse)
    print("rebuilding drive 2 byte {}; {:02x} = gf_mul( {:02x}, {:02x})".format(i,drive[2][i],partialQ,inverse))
    #dump_table("drive[2]",drive[2])   

print ("\nafter reconstructing data drive")
dump_all_drives()
    
# rebuild P
generateP()

print("\nafter reconstructing p drive")
dump_all_drives()
    
                    

after failing drives 2 and P
--- drive0 ---
0x66, 0x69, 0x72, 0x73, 0x74, 
--- drive1 ---
0x73, 0x65, 0x63, 0x6e, 0x64, 
--- drive2 ---
0x00, 0x00, 0x00, 0x00, 0x00, 
--- drive3 ---
0x66, 0x6f, 0x72, 0x74, 0x68, 
--- P parity ---
0x00, 0x00, 0x00, 0x00, 0x00, 
--- Q parity ---
0x5a, 0x41, 0xba, 0xfd, 0x56, 

rebuilding drive 2 byte 0; 74 = gf_mul( cd, 47)
rebuilding drive 2 byte 1; 68 = gf_mul( bd, 47)
rebuilding drive 2 byte 2; 69 = gf_mul( b9, 47)
rebuilding drive 2 byte 3; 72 = gf_mul( d5, 47)
rebuilding drive 2 byte 4; 64 = gf_mul( 8d, 47)

after reconstructing data drive
--- drive0 ---
0x66, 0x69, 0x72, 0x73, 0x74, 
--- drive1 ---
0x73, 0x65, 0x63, 0x6e, 0x64, 
--- drive2 ---
0x74, 0x68, 0x69, 0x72, 0x64, 
--- drive3 ---
0x66, 0x6f, 0x72, 0x74, 0x68, 
--- P parity ---
0x00, 0x00, 0x00, 0x00, 0x00, 
--- Q parity ---
0x5a, 0x41, 0xba, 0xfd, 0x56, 

after reconstructing p drive
--- drive0 ---
0x66, 0x69, 0x72, 0x73, 0x74, 
--- drive1 ---
0x73, 0x65, 0x63, 0x6e, 0x64, 
--- drive2 ---


In [26]:
# Fail and reconstruct two data drives
drive[1] = [0]*5
drive[2] = [0]*5
print("after failing drives 1 and 2")
dump_all_drives()
print()

#Rebuild the data for drive[1] using Galois divisors
for i in range(0, blocksize):
    partialP = 0
    partialQ = 0
    for d in range(0,numdrives):
        partialP = gf_add(partialP,drive[d][i])
        partialQ = gf_add(gf_mul(gfilog[d],drive[d][i]),
                          partialQ)

    # Calculations here proceed a bit differently from the Anvin paper.
    
    #We calculate the (P + Pxy) and (Q + Qxy) terms:
    partialP = gf_add(partialP,driveP[i])
    partialQ = gf_add(partialQ,driveQ[i])
    
 
    # Now we calculate a partial sum 'assuming' there was just one missing drive (2nd)
    mid = gf_add(gf_mul(gfilog[2],partialP), 
                 partialQ)
    
    #Generate the GF divisor that sums the two missing drives
    g = gf_div(1,(gf_add(gfilog[1],gfilog[2]))) 
    
    #Divide the "partial sum" by the syndrome for both missing drives. This eliminates the second drive and leaves the first.
    data=gf_mul(mid,g)
    drive[1][i] = data
    print("rebuilding byte {} of drive[1], {:02x} = gf_mul({:02x},{:02x})".format(i,data,mid,g))

dump_all_drives()
print()

# Now we can easily rebuild the data for the second missing drive with simple xor
for i in range(0,blocksize):
    drive[2][i] = gf_add(drive[0][i],drive[1][i],drive[3][i],driveP[i])
    print("rebuilding byte {} of drive[2], {:02x} = gf_add({:02x},...{:02x})".format(i,drive[2][i],drive[0][i],driveP[i]))
    
dump_all_drives()
            

after failing drives 1 and 2
--- drive0 ---
0x66, 0x69, 0x72, 0x73, 0x74, 
--- drive1 ---
0x00, 0x00, 0x00, 0x00, 0x00, 
--- drive2 ---
0x00, 0x00, 0x00, 0x00, 0x00, 
--- drive3 ---
0x66, 0x6f, 0x72, 0x74, 0x68, 
--- P parity ---
0x07, 0x0b, 0x0a, 0x1b, 0x1c, 
--- Q parity ---
0x5a, 0x41, 0xba, 0xfd, 0x56, 

rebuilding byte 0 of drive[1], 73 = gf_mul(37,7a)
rebuilding byte 1 of drive[1], 65 = gf_mul(43,7a)
rebuilding byte 2 of drive[1], 63 = gf_mul(57,7a)
rebuilding byte 3 of drive[1], 6e = gf_mul(79,7a)
rebuilding byte 4 of drive[1], 64 = gf_mul(45,7a)
--- drive0 ---
0x66, 0x69, 0x72, 0x73, 0x74, 
--- drive1 ---
0x73, 0x65, 0x63, 0x6e, 0x64, 
--- drive2 ---
0x00, 0x00, 0x00, 0x00, 0x00, 
--- drive3 ---
0x66, 0x6f, 0x72, 0x74, 0x68, 
--- P parity ---
0x07, 0x0b, 0x0a, 0x1b, 0x1c, 
--- Q parity ---
0x5a, 0x41, 0xba, 0xfd, 0x56, 

rebuilding byte 0 of drive[2], 74 = gf_add(66,...07)
rebuilding byte 1 of drive[2], 68 = gf_add(69,...0b)
rebuilding byte 2 of drive[2], 69 = gf_add(72,...0a)


In [ ]:
def full_stripe_write(start_byte, data, data_drives, failed_drives):
    stripe_size = len(data_drives)
    if start_byte % stripe_size != 0:
        print("Starting byte must be a multiple of stripe size.")
        return
    
    for i in range(start_byte, start_byte + len(data), stripe_size):
        stripe_data = data[i:i + stripe_size]
        for drive_num in range(stripe_size):
            if drive_num not in failed_drives:
                data_drives[drive_num][i // stripe_size] = stripe_data[drive_num]
        
        generateP(data_drives, failed_drives)
        generateQ(data_drives, failed_drives)


These changes should address the limitations and make the RAID-6 simulator more flexible and functional. You can integrate these functions into your existing code, and make sure to test them thoroughly to ensure their correctness and functionality.